In [28]:
import numpy as np
import pandas as pd
import scipy.stats as st

from statsmodels.tsa.api import VAR

In [4]:
def Z_matrix(y: np.array, p: int, c: int):
    """Give the Z matrix for a given input

    Args:
        y (np.array): _description_
        p (int): _description_
        c (int): _description_

    Returns:
        _type_: _description_
    """
    
    K = np.size(y, 0)
    T = np.size(y, 1) - p
    
    Z = y[:, :T]
    
    for i in range(1, p):
        Z = np.concatenate((y[:,i:T+i], Z), 0)
        
    if (c ==1):
        Z = np.concatenate((np.ones(shape=(1, T)), Z), 0)
       
    print('dimensions of Z-matrix [(K*p)+1 x T]:')   
    print(np.size(Z,0))
    print(np.size(Z,1))         
    
    return Z

In [5]:
def B_matrix(y: np.array, p: int, c: int):
    """_summary_

    Args:
        y (np.array): _description_
        p (int): _description_
        c (int): _description_

    Returns:
        _type_: _description_
    """
    
    Z = Z_matrix(y, p, c)
    
    y = y[:,p:]
    
    B = y @ Z.T @ np.linalg.inv((Z@Z.T))
    
    return B

In [96]:
def granger(y: np.array, p: int, dummy_vec: list, c=1):
    """_summary_

    Args:
        data (pd.DataFrame): _description_
        p (int): _description_
        dummy (int): _description_

    Returns:
        _type_: Wald- and F-statistic together with implied p-values
    """
    
    y = y.T
    n_cause = sum(dummy_vec)
    n_caused = len(dummy_vec) - n_cause
       
    # arrange in right order
    cause = []
    caused = []
    for i, n in enumerate(dummy_vec):
        if n == 1:
            cause.append(y[i])
        else:
            caused.append(y[i])
    
    cause = np.column_stack(cause)
    caused = np.column_stack(caused)
    y = np.concatenate((cause, caused), axis=1)
        
    # get B matrix
    K = y.shape[1]  # number of variables
    B = B_matrix(y, p, c)
    
    vec_B = B.flatten(order="F").T
    vec_B_wo_interc = vec_B[K:]
    


    #lamba_w = (C@vec_B).T @ (C@(np.linalg.inv(Z@Z.T)))
        
    
    
    return 

In [47]:
# read in data
awm = pd.read_csv("awm19up18.csv")
awm.rename(columns={awm.columns[0]: "Q" }, inplace = True)

of_interest = ["Q", "YER", "ITR", "LTN", "STN"]
awm = awm[awm.columns.intersection(of_interest)]
awm.set_index('Q', inplace=True)

awm.head()

,YER,ITR,STN,LTN
Q,,,,
1970Q1,738304.250471,191787.320701,7.986993,7.922865
1970Q2,752495.866789,203176.720610,7.956545,8.254439
1970Q3,761561.610862,206267.925392,7.602189,8.384747
1970Q4,770787.033957,205426.358549,7.242308,8.340932
1971Q1,769439.321026,204324.652554,6.516898,7.860624


In [48]:
awm["YER_log"] = np.log(awm['YER'])
awm["ITR_log"] = np.log(awm['ITR'])

awm["d_lgdp"] = awm["YER_log"].diff()
awm["d_invest"] = awm["ITR_log"].diff()

awm["d_lgdp"] = awm["d_lgdp"] * 400
awm["d_invest"] = awm["d_invest"] * 400

awm["d_R"] = awm["LTN"].diff()
awm["d_r"] = awm["STN"].diff()

awm.dropna(inplace=True)

awm.head()

,YER,ITR,STN,LTN,YER_log,ITR_log,d_lgdp,d_invest,d_R,d_r
Q,,,,,,,,,,
1970Q2,752495.866789,203176.720610,7.956545,8.254439,13.531151,12.221831,7.615800,23.075637,0.331574,-0.030448
1970Q3,761561.610862,206267.925392,7.602189,8.384747,13.543126,12.236931,4.790229,6.039915,0.130308,-0.354356
1970Q4,770787.033957,205426.358549,7.242308,8.340932,13.555167,12.232843,4.816415,-1.635326,-0.043815,-0.359880
1971Q1,769439.321026,204324.652554,6.516898,7.860624,13.553417,12.227465,-0.700008,-2.150982,-0.480308,-0.725410
1971Q2,779295.962146,209724.713360,5.938638,7.930762,13.566146,12.253551,5.091522,10.434248,0.070138,-0.578261


In [60]:
y_t = np.array(awm[["d_lgdp", "d_invest", "d_R", "d_r"]])
np.shape(y_t)


(191, 4)

In [97]:
granger(y_t, 2, [0, 0, 1, 1], 1)

dimensions of Z-matrix [(K*p)+1 x T]:
9
189


(36,)

In [98]:
# check result with statsmodels VAR module
model = VAR(awm[["d_lgdp", "d_invest", "d_R", "d_r"]])
results = model.fit(2)

c:\Users\jonas\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


Test statistic,Critical value,p-value,df
25.79,15.51,0.001,8


In [ ]:
granger_stat_wald = results.test_causality(['d_R', "d_r"], ["d_lgdp", "d_invest"], kind='wald')
granger_stat_wald.summary()

In [ ]:
granger_stat_f = results.test_causality(['d_R', "d_r"], ["d_lgdp", "d_invest"], kind='f')
granger_stat_f.summary()